In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Generate synthetic stock price data for illustration purposes
# Replace this with your actual data loading and preprocessing
np.random.seed(42)
data = np.random.randint(50, 150, size=1000) + np.cumsum(np.random.normal(0, 1, size=1000))
df = pd.DataFrame(data, columns=['Closing_Price'])
df

In [ ]:
# Data preprocessing
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['Closing_Price']])

# Create sequences for the CNN model
sequence_length = 10
X, y = [], []
for i in range(len(df_scaled) - sequence_length):
    X.append(df_scaled[i : i + sequence_length])
    y.append(df_scaled[i + sequence_length, 0])

X, y = np.array(X), np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))         
model.add(LSTM(50, activation='relu',return_sequences=True))
model.add(LSTM(50, activation='relu',return_sequences=True))
model.add(LSTM(50, activation='relu',return_sequences=True))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
# Compile the CNN model
model.compile(optimizer='adam', loss='mse')

# Train the CNN model
model.fit(X_train, y_train, epochs=100, batch_size=20, validation_data=(X_test, y_test))

# Make predictions using the CNN model
cnn_predictions = model.predict(X_test)

# Build the Random Forest model
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)

# Flatten the CNN predictions for input to the RF model
X_rf = cnn_predictions.flatten().reshape(-1, 1)

# Train the Random Forest model
rf_model.fit(X_rf, y_test)

# Make predictions using the Random Forest model
rf_predictions = rf_model.predict(X_rf)

# Combine CNN and RF predictions with error compensation using XGBoost
combined_predictions = cnn_predictions.flatten() + rf_predictions

# Train an XGBoost model for error compensation
xgb_model = XGBRegressor(n_estimators=1000, random_state=42)

# Train the XGBoost model on the absolute errors
absolute_errors = np.abs(y_test - combined_predictions)
xgb_model.fit(X_rf, absolute_errors)

# Make predictions using the XGBoost model
xgb_predictions = xgb_model.predict(X_rf)

# Adjust combined predictions using XGBoost predictions
final_predictions = combined_predictions + xgb_predictions

# Inverse transform the predictions for plotting
final_predictions = scaler.inverse_transform(np.hstack((final_predictions.reshape(-1, 1), np.zeros_like(final_predictions).reshape(-1, 1))))[:, 0]

# Calculate and print the Mean Squared Error (MSE) as an evaluation metric
mse = mean_squared_error(y_test, final_predictions)
print(f'Mean Squared Error (MSE): {mse}')

# Plotting
plt.figure(figsize=(15, 6))
plt.plot(df.index[-len(final_predictions):], df['Closing_Price'].values[-len(final_predictions):], label='True Price')
plt.plot(df.index[-len(final_predictions):], final_predictions, label='Final Predicted Price', linestyle='dashed')
plt.title('Stock Price Prediction with CNN, RF, and XGBoost Error Compensation')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()


In [ ]:
final_predictions

In [ ]:
df['Closing_Price'].values[-len(final_predictions):]